In [62]:
import numpy as np
import pandas as pd

In [63]:
df = pd.read_csv('goal_templates.csv')

In [64]:
# Clean the documents. 
# In this case it is remove spaces after commas and lowercase the header
df.columns = [column.strip().lower() for column in df.columns]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

##### Connect to MongoDB


In [65]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['kavigai_rag']
collection = db['goal_templates']


#### Convert documents in DataFrame to Dictionary and Insert them into the kavigai_rag vector databse


In [74]:
# docu_dict = df.to_dict('records')
# collection.insert_many(docu_dict);

#### Create embeddings for the documents and Insert them into the kavigai_rag vector databse

In [75]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Retrieve documents from vector databse
documents = collection.find()
goal_templates_embeddings = []

# Generate embeddings and store them in vector databse
for doc in documents:
    goal_template_name = doc.get('goal_template', '') # use get() to avoid error when field is missing
    description = doc['description']  # will be an error when the field is missing
    price = doc.get('price', '')
    category = doc.get('category', '')
    
    # Concatenate title and content (or other fields)
    combined_text = f"{goal_template_name}. {description}, {price}. {category}"
    embedding = model.encode(combined_text)
    goal_templates_embeddings.append({'_id': doc['_id'], 'goal_templates_embeddings': embedding.tolist()})

# Insert embeddings into a separate collection
goal_templates_embedding_collection = db['goal_templates_embeddings']
# goal_templates_embedding_collection.insert_many(goal_templates_embeddings)
print("\nDocument embeddings inserted successfully! {documents}")


c:\Users\a\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Document embeddings inserted successfully! {documents}


In [77]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['kavigai_rag']
collection = db['goal_templates']


In [80]:
from scipy.spatial.distance import cdist
import numpy as np

def retrieve_documents(query, top_k=3):
    # Generate query embedding
    query_embedding = model.encode(query).reshape(1, -1)

    # Retrieve all embeddings collection from goal_templates_embedding_collection collection
    embeddings = list(goal_templates_embedding_collection.find())

    # Compute cosine similarity between query embedding and document embeddings
    distances = cdist(query_embedding, np.array([np.array(doc['goal_templates_embeddings']) for doc in embeddings]), 'cosine')

    # Find the top_k (top 3 in this case) closest documents
    closest_docs = np.argsort(distances[0])[:top_k]
    doc_ids = [embeddings[idx]['_id'] for idx in closest_docs]

    # Retrieve full documents corresponding to embeddings from kavigai_rag databse
    retrieved_docs = collection.find({"_id": {"$in": doc_ids}})
    return list(retrieved_docs)

In [33]:
import os
import openai
from openai import OpenAI

OAIclient = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


In [36]:

# Function to retrieve relevant documents from MongoDB using embeddings
def generate_response(query, documents):
    # Prepare the prompt for OpenAI
    prompt = f"Query: {query}\n\n"
    prompt += "Here are some relevant documents:\n"
    print(f'prompt: {prompt}')

    for doc in documents:
        prompt += f"\nGoal Template: {doc['goal template']}\nDescription: {doc['description']}\n"
        print(f'prompt: {prompt}')

    # Send the query to OpenAI
    response = openai.Completion.create(
        engine="text-ada-001",
        # engine="gpt-4o-mini-2024-07-18",
        prompt=prompt,
        max_tokens=20
    )

    return response['choices'][0]['text']


In [32]:
from flask import Flask, render_template, request

# Main route for the web interface
# Initialize Flask app
app = Flask(__name__)
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Get the query from the user input
        query = request.form['query']

        # Retrieve relevant documents
        documents = retrieve_documents(query)

        # Generate a response using the OpenAI API
        response = generate_response(query, documents)

        return render_template('index.html', query=query, response=response, documents=documents)
    
    return render_template('index.html', query='', response='', documents=[])

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

In [33]:
%tb

SystemExit: 1